In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm.auto import tqdm
from statsmodels.stats.outliers_influence import variance_inflation_factor
from itertools import product

from scripts.feature_engineering import add_base_features, create_df
from scripts.utils import save_fig
%reload_ext autoreload
%autoreload 2
plt.rcParams['figure.figsize'] = (12, 4)
font = {'size'   : 18}
plt.rc('font', **font)

In [ ]:
LIST = ['ERA5', 'METEO-FRANCE']
WEATHER_SOURCE = LIST[1]

In [ ]:
OPTIONS = ['per_vineyard', 'per_region']
OPTION = OPTIONS[0]

In [ ]:
LAST_YEAR = 2022

## Load prices

In [ ]:
prices = pd.read_excel('data/prices/prices_per_vineyard.xlsx', index_col=0)
prices.tail()

## Load weather

In [ ]:
locations = {
    "Médoc": [45.12, -0.74],
    "Saint-Emilion": [44.8, -0.192],
    "Pomerol": [44.925, -0.198],
    "Pessac-Léognan": [44.81, -0.62],
}
locations = pd.read_excel(
    "data/gathered_from_internet/vineyard_locations.xlsx"
).set_index("name")

In [ ]:
def find_closest_in_list(distances):
    min_list = distances[0]
    min_index = 0
    for i in range(1, len(distances)):
        if distances[i] < min_list:
            min_list = distances[i]
            min_index = i
    return min_index


def find_closest_points(coords, dataset):
    lon, lat = coords
    lons = dataset.variables["longitude"][:]
    lats = dataset.variables["latitude"][:]

    lon_distances = [np.abs(lon - lons[i]) for i in range(len(lons))]
    lat_distances = [np.abs(lat - lats[i]) for i in range(len(lats))]
    closest_lon = lons[find_closest_in_list(lon_distances)]
    closest_lat = lats[find_closest_in_list(lat_distances)]
    return closest_lon, closest_lat

In [ ]:
top_wines = list(locations.keys())

dict_weather_by_vineyard = {}
pheno = pd.read_excel("data/pheno/generated_pheno.xlsx", index_col=0)

if WEATHER_SOURCE == "ERA5":
    from netCDF4 import Dataset
    import xarray
    import glob

    p = xarray.merge([xarray.open_dataset(f) for f in glob.glob("data/ERA5/p/*.nc")])
    skt = xarray.merge(
        [xarray.open_dataset(f) for f in glob.glob("data/ERA5/skt/*.nc")]
    )
    evap = xarray.merge(
        [xarray.open_dataset(f) for f in glob.glob("data/ERA5/evap/*.nc")]
    )
    print(p)
    print(skt)
    print(evap)

    dict_chosen_lon_lat = {}
    CONVERT_K_TO_C = 273.15
    for vineyard in tqdm(top_wines):
        print(vineyard)
        target_lat, target_lon = (
            locations.loc[vineyard, "latitude"],
            locations.loc[vineyard, "longitude"],
        )

        method = "linear"
        df_p = (
            p["tp"]
            .interp(longitude=target_lon, latitude=target_lat, method=method)
            .to_dataframe()
        )
        chosen_lon, chosen_lat = df_p.iloc[0, :][["longitude", "latitude"]]
        print("Chosen coordinates:", chosen_lon, chosen_lat)
        df_p["date"] = pd.to_datetime(df_p.index.date)

        df_weather = (
            df_p.groupby("date").sum().copy().rename(columns={"tp": "P"}) * 1000
        )

        df_skt = (
            skt["skt"]
            .interp(longitude=target_lon, latitude=target_lat, method=method)
            .to_dataframe()
        )
        df_skt["date"] = pd.to_datetime(df_skt.index.date)

        df_weather["Tn"] = df_skt.groupby("date").min()["skt"].copy() - CONVERT_K_TO_C
        df_weather["Tx"] = df_skt.groupby("date").max()["skt"].copy() - CONVERT_K_TO_C
        df_weather["Tm"] = df_skt.groupby("date").mean()["skt"].copy() - CONVERT_K_TO_C

        df_evap = (
            evap["pev"]
            .interp(longitude=target_lon, latitude=target_lat, method=method)
            .to_dataframe()
        )
        df_evap["date"] = pd.to_datetime(df_evap.index)

        df_weather["ETP"] = -df_evap.groupby("date").sum()["pev"].copy() * 1000
        df_weather.index = pd.to_datetime(df_weather.index)
        dict_weather_by_vineyard[vineyard] = df_weather.copy()
        dict_chosen_lon_lat[vineyard] = chosen_lon, chosen_lat

elif WEATHER_SOURCE == "METEO-FRANCE":
    weather_st_em = pd.read_csv(
        "data/weather/generated_weather_st_em.csv", parse_dates=["Date"]
    ).set_index("Date")
    weather_medoc = pd.read_csv(
        "data/weather/generated_weather_pauillac.csv", parse_dates=["Date"]
    ).set_index("Date")
    weather_graves = pd.read_csv(
        "data/weather/generated_weather_leognan.csv", parse_dates=["Date"]
    ).set_index("Date")
    dict_weather_by_region = {
        "Médoc": weather_medoc,
        "Saint-Emilion": weather_st_em,
        "Pomerol": weather_st_em,
        "Pessac-Léognan": weather_graves,
    }

In [ ]:
top_wines = list(prices["Vineyard"].unique())
region_per_vineyard = {
    vineyard: prices.loc[prices["Vineyard"] == vineyard, "Appellation"].unique()[0]
    for vineyard in top_wines
}
all_dfs = []
for vineyard in tqdm(top_wines):
    df_weather_vineyard = create_df(
        add_base_features(dict_weather_by_region[region_per_vineyard[vineyard]]), pheno
    )
    df_elements = df_weather_vineyard.loc[:LAST_YEAR].copy()
    interpolated_values = (
        prices.loc[prices["Vineyard"] == vineyard, ["Vintage", "Price"]]
        .set_index("Vintage")
        .values
    )
    df_elements["0 - Price"] = np.concatenate([interpolated_values.ravel(), [np.nan, np.nan]]) # empty values for last 2 years
    all_dfs.append(df_elements)

df_agg = pd.concat(all_dfs, axis=1)
features = list(df_weather_vineyard.columns)
prod = list(product(top_wines, features + ["0 - Price"]))
df_agg.columns = pd.MultiIndex.from_tuples(prod)
display(df_agg.head())

In [ ]:
df_agg['Château Angélus']['0 - Price'].tail(10)

In [ ]:
df_agg.to_csv(f"data/features/features-{WEATHER_SOURCE}_per_vineyard.csv")

### Variance inflation factor

In [ ]:
example='Petrus'

In [ ]:
a = df_agg[example][
    ["WD: flowering - harvest", "P: flowering", "DTR: véraison - harvest"]
]

variables = list(range(a.shape[1]))
vif = [variance_inflation_factor(a.iloc[:, variables].values, ix) for ix in variables]
print("VIF for the three variables is: ", vif)

### Correlations

In [ ]:
features = [feature for feature in features if 
    'Tn' not in feature
    and 'Tx' not in feature
    and 'ETP' not in feature
]

In [ ]:
correlations = (
    df_agg.loc[1990:2013, ([example], features + ["0 - Price"])].T.droplevel(0).corr()
)

In [ ]:
correlations.loc[:, '0 - Price'].sort_values(ascending=False)

In [ ]:
plt.figure(figsize=(14, 14))
cmap = sns.diverging_palette(250, 10, as_cmap=True)
sns.heatmap(
    df_agg.loc[1990:, ([example], features + ["0 - Price"])].T.droplevel(0).T.corr(),
    cmap=cmap,
)
plt.tight_layout()
plt.savefig("views/heatmap.png")
plt.show()

## Plot temperature evolution

In [ ]:
tm_st_em = dict_weather_by_region['Saint-Emilion']['Tm']
tm_st_em = tm_st_em[tm_st_em.index.month.isin(range(4, 10))].reset_index()
tm_yearly_st_em = tm_st_em.groupby(tm_st_em['Date'].dt.year).mean()
tm_yearly_st_em.head()

In [ ]:
plt.rc("text", usetex=False)
plt.figure(figsize=(10, 5))
font = {"weight": "regular", "family": "serif", "size": 16}
plt.grid(True, which="both", axis="both", alpha=0.5)

graycolors = sns.mpl_palette("Greys_r", 4)
plt.rc("font", **font)
plt.rc("xtick", labelsize="14")
plt.rc("ytick", labelsize="14")

tm_yearly_st_em["Year"] = tm_yearly_st_em.index
sns.lineplot(
    data=tm_yearly_st_em,
    x="Year",
    y="Tm",
    marker="o",
    markersize=6,
    linewidth=1.5,
    color="black",
    markeredgecolor="k",
    linestyle="dashed",
)
plt.gca().lines[0].set_linestyle("--")
plt.ylabel("Temperature (°C)")
plt.ylim((14.8, 20))
save_fig("views/temperature", "single")
plt.show()

---
# End of notebook